In [ ]:
### Parikshit Sunil Deshmukh - pdeshmuk - 50247649
### Mahalakshmi Padma Sri Harsha Maddu - mmaddu - 50246769

In [129]:
#Referances:
# https://towardsdatascience.com/multi-class-text-classification-with-pyspark-7d78d022ed35

from pyspark.sql import SQLContext
from pyspark import SparkContext
sc =SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

##Importing the data

data = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('/media/parik/New Volume/DIC/lab3/Final_Data/Final_Data/output.csv')


In [130]:
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml.classification import LogisticRegression

##Tokenizer
regxTokenizer = RegexTokenizer(inputCol="Article", outputCol="words", pattern="\\W")

##StopWords

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
stop_words = set(stopwords.words('english'))
stop_words.add("advertisement")
stop_words.add("said")
stop_words.add("by")
stop_words.add("")
stop_words.add(" ")
stop_words.add("  ")
stop_words.add("mr")
stop_words = list(stop_words)
newL=["advertisement","percent","space","years","another","first","second","third","one","might","two","three","think","right","better","people","women","family","person","trump","woman","article","state","still","including","college","united","states","new","york","later","better","president","office","working","white","house","chief","start","small","world","ourselves", "hers", "between", "yourself", "but", "again", "there", "about", "once", "during", "out", "very", "having", "with", "they", "own", "an", "be", "some", "for", "do", "its", "yours", "such", "into", "of", "most", "itself", "other", "off", "is", "s", "am", "or", "who", "as", "from", "him", "each", "the", "themselves", "until", "below", "are", "we", "these", "your", "his", "through", "don", "nor", "me", "were", "her", "more", "himself", "this", "down", "should", "our", "their", "while", "above", "both", "up", "to", "ours", "had", "she", "all", "no", "when", "at", "any", "before", "them", "same", "and", "been", "have", "in", "will", "on", "does", "yourselves", "then", "that", "because", "what", "over", "why", "so", "can", "did", "not", "now", "under", "he", "you", "herself", "has", "just", "where", "too", "only", "myself", "which", "those", "i", "after", "few", "whom", "t", "being", "if", "theirs", "my", "against", "a", "by", "doing", "it", "how", "further", "was", "here", "than"]
for stp in newL:
    stop_words.append(stp)

stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered").setStopWords(stop_words)

##Bag of words approach: CountVectors

countVectors = CountVectorizer(inputCol="filtered", outputCol="features", vocabSize=10000, minDF=5)

In [131]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

###String Indexer

label_stringIdx = StringIndexer(inputCol = "Category", outputCol = "label")

#Creating
pipeline = Pipeline(stages=[regxTokenizer, stopwordsRemover, countVectors, label_stringIdx])
pipelineFit = pipeline.fit(data)
dataset = pipelineFit.transform(data)
dataset.select('label').distinct().show()
dataset.show()

+-----+
|label|
+-----+
|  0.0|
|  1.0|
|  3.0|
|  2.0|
+-----+

+--------------------+--------+--------------------+--------------------+--------------------+-----+
|             Article|Category|               words|            filtered|            features|label|
+--------------------+--------+--------------------+--------------------+--------------------+-----+
|australia office ...|Politics|[australia, offic...|[australia, sex, ...|(1519,[5,8,9,14,1...|  2.0|
|italy artists jas...|Politics|[italy, artists, ...|[italy, artists, ...|(1519,[1,5,8,12,1...|  2.0|
|art design notebo...|Politics|[art, design, not...|[art, design, not...|(1519,[1,5,8,10,1...|  2.0|
|region actress go...|Politics|[region, actress,...|[region, actress,...|(1519,[5,7,10,14,...|  2.0|
|byop ed contribut...|Politics|[byop, ed, contri...|[byop, ed, contri...|(1519,[1,2,4,8,9,...|  2.0|
|books abortion bo...|Politics|[books, abortion,...|[books, abortion,...|(1519,[1,2,5,8,10...|  2.0|
|borderfrom museum...|Poli

In [143]:

###Splitting the data into train and test set

(trainingsplit, testsplit) = dataset.randomSplit([0.7, 0.3], seed = 100)
print("Number of Training set: " + str(trainingsplit.count()))
print("Number of Test set: " + str(testsplit.count()))

Number of Training set: 81
Number of Test set: 42


In [144]:

####Naive Bayes Model

from pyspark.ml.classification import NaiveBayes

###Creating the model
naiveBayesModel= NaiveBayes(smoothing=1)

###Fitting the model with train set
model = naiveBayesModel.fit(trainingsplit)

##Predicting the output for test set
predictions_output = model.transform(testsplit)
type(predictions_output)

predictions.registerTempTable('out')


In [145]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

##Evaluating out predicted output for accuracy 

evaluating = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluating.evaluate(predictions)

0.8784940984940983

In [146]:
####Logisitc Regression Model without TFIDF

## Craeting Logistic Regression Model
logisticRegressionModel = LogisticRegression(maxIter=20, regParam=0.8, elasticNetParam=0)

## Fitting the model with training set 
lrModel = logisticRegressionModel.fit(trainingsplit)

## Predicting the output for the test set
predictions1_output = lrModel.transform(testsplit)



predictions1_output.registerTempTable('out1')

##Wrong predictions:

sqlContext.sql('select *  from out1 where label<>prediction').show()



+--------------------+--------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+----------+
|             Article|Category|               words|            filtered|            features|label|       rawPrediction|         probability|prediction|
+--------------------+--------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+----------+
|book review nonfi...|Business|[book, review, no...|[book, review, no...|(1519,[1,2,3,5,8,...|  0.0|[-1.0407086815508...|[0.06865558858659...|       1.0|
|byop ed contribut...|Business|[byop, ed, contri...|[byop, ed, contri...|(1519,[2,3,5,15,1...|  0.0|[-0.2716502288934...|[0.14468217128994...|       2.0|
|byop ed obamaby j...|Business|[byop, ed, obamab...|[byop, ed, obamab...|(1519,[1,2,3,8,14...|  0.0|[-0.5333269610217...|[0.09317109903603...|       2.0|
|bythe stonehow de...|  Movies|[bythe, stonehow,...|[bythe, stonehow,...|(15

In [119]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

### Evaluting the predictions
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

0.8784940984940983

In [149]:
from pyspark.ml.feature import HashingTF, IDF

##Using TF-IDF for logistic regression


hTF = HashingTF(inputCol="filtered", outputCol="rawFeatures", numFeatures=10000)
idf = IDF(inputCol="rawFeatures", outputCol="features", minDocFreq=5) #minDocFreq: remove sparse terms
pipeline1 = Pipeline(stages=[regxTokenizer, stopwordsRemover, hTF, idf, label_stringIdx])
pFit = pipeline1.fit(data)
data_set = pFit.transform(data)

data_set.show(3)
data_set.registerTempTable('dataset')


+--------------------+--------+--------------------+--------------------+--------------------+--------------------+-----+
|             Article|Category|               words|            filtered|         rawFeatures|            features|label|
+--------------------+--------+--------------------+--------------------+--------------------+--------------------+-----+
|australia office ...|Politics|[australia, offic...|[australia, sex, ...|(10000,[1,70,88,2...|(10000,[1,70,88,2...|  2.0|
|italy artists jas...|Politics|[italy, artists, ...|[italy, artists, ...|(10000,[69,77,110...|(10000,[69,77,110...|  2.0|
|art design notebo...|Politics|[art, design, not...|[art, design, not...|(10000,[69,77,110...|(10000,[69,77,110...|  2.0|
+--------------------+--------+--------------------+--------------------+--------------------+--------------------+-----+
only showing top 3 rows



In [150]:
sqlContext.sql('select features from dataset ').show()

+--------------------+
|            features|
+--------------------+
|(10000,[1,70,88,2...|
|(10000,[69,77,110...|
|(10000,[69,77,110...|
|(10000,[41,60,219...|
|(10000,[21,47,76,...|
|(10000,[8,26,87,1...|
|(10000,[68,78,87,...|
|(10000,[2,103,132...|
|(10000,[9,87,93,1...|
|(10000,[15,21,90,...|
|(10000,[68,78,87,...|
|(10000,[9,87,93,1...|
|(10000,[8,26,87,1...|
|(10000,[41,60,219...|
|(10000,[69,77,110...|
|(10000,[23,37,59,...|
|(10000,[23,37,59,...|
|(10000,[69,77,110...|
|(10000,[1,8,21,60...|
|(10000,[68,76,132...|
+--------------------+
only showing top 20 rows



In [154]:
###Logistic regression model with TF-IDF

## Train test split
(trainingsplit, testsplit) = data_set.randomSplit([0.7, 0.3], seed = 100)

##Creating the model
logisticRegressionModel = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)

###Fitting the model for train set
lrModel = logisticRegressionModel.fit(trainingsplit)

### Testing the model for test set
predictions = lrModel.transform(testsplit)
predictions.registerTempTable('logTable')

predictions.filter(predictions['prediction'] == 0) \
    .select("Article","Category","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)

+------------------------------+--------+------------------------------+-----+----------+
|                       Article|Category|                   probability|label|prediction|
+------------------------------+--------+------------------------------+-----+----------+
|imageone wework spaces manh...|Business|[0.9897503661665389,0.00580...|  0.0|       0.0|
|bybusiness pressapril hope ...|Business|[0.9784120703904606,0.00709...|  0.0|       0.0|
|bybusiness pressapril growt...|Business|[0.9768403506839897,0.01026...|  0.0|       0.0|
|business day business highl...|Business|[0.9695175534827642,0.01236...|  0.0|       0.0|
|business day business highl...|Business|[0.967015553130405,0.011364...|  0.0|       0.0|
|business day bull mobile ou...|Business|[0.9591095118497532,0.01895...|  0.0|       0.0|
|business day cohn departure...|Business|[0.9529455876084199,0.01579...|  0.0|       0.0|
|weddings business pleasure ...|Business|[0.9391091853882036,0.02723...|  0.0|       0.0|
|bybusines

In [155]:
sqlContext.sql('select * from logTable where label<>prediction').show()

+--------------------+--------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+----------+
|             Article|Category|               words|            filtered|         rawFeatures|            features|label|       rawPrediction|         probability|prediction|
+--------------------+--------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+----------+
|book review nonfi...|Business|[book, review, no...|[book, review, no...|(10000,[76,174,20...|(10000,[76,174,20...|  0.0|[-0.0603265388179...|[0.23203750947269...|       2.0|
|byop ed contribut...|Business|[byop, ed, contri...|[byop, ed, contri...|(10000,[47,76,118...|(10000,[47,76,118...|  0.0|[-0.0701328524539...|[0.19025038455620...|       2.0|
|byop ed obamaby j...|Business|[byop, ed, obamab...|[byop, ed, obamab...|(10000,[15,45,110...|(10000,[15,45,110...|  0.0|[-0.

In [156]:
### Evaluting the performance 

evaluating = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluating.evaluate(predictions)

0.8572486772486771